# Installing Packages

In [1]:
# pip install pandas sqlalchemy pyodbc


# Create Connection

In [ ]:
import pandas as pd
import os
import glob
from tqdm import tqdm
from sqlalchemy import create_engine

# -------------------------------
# PostgreSQL Connection Settings
# -------------------------------
user = ""          # your pgAdmin username
password = "" # replace with your PostgreSQL password
host = "localhost"         # if running locally
port = 5432                # default PostgreSQL port
database = "IPEDSDataWarehouse"      # the DB you created in pgAdmin

# -------------------------------
# SQLAlchemy Connection String
# -------------------------------
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"

# -------------------------------
# Create Engine
# -------------------------------
engine = create_engine(connection_string)

print("✅ Connected successfully to PostgreSQL")


# Set Base Folder

In [ ]:
base_folder = r'C:\Users\3D\OneDrive\Desktop\IPEDS Final'
file_paths = glob.glob(os.path.join(base_folder, "**", "*.csv"), recursive=True)

print(f"📂 Found {len(file_paths)} CSV files to process")  



# Previewing a few paths

In [ ]:
for f in file_paths:
    print("➡️", f)

# Creating All Tables 

In [ ]:
import chardet

for file_path in file_paths:
    table_name = os.path.basename(file_path).replace('.csv','').replace('.', '_').replace('-', '_').lower()
    
    # Detect encoding from first 100 KB
    with open(file_path, 'rb') as f:
        rawdata = f.read(100000)
        detected = chardet.detect(rawdata)
        encoding = detected['encoding'] if detected['encoding'] not in [None, 'ascii'] else 'latin1'

    # Read only header safely
    df_sample = pd.read_csv(file_path, nrows=0, encoding=encoding)

    # Create table in SQL Server
    df_sample.to_sql(
        name=table_name,
        con=engine,
        schema='Bronze',
        if_exists='replace',
        index=False
    )
    print(f"✅ Table created: {table_name}")


# Checking Total Tables Created

In [ ]:
import psycopg2

# Connect using psycopg2
conn = psycopg2.connect(
    dbname='IPEDSDataWarehouse',
    user='',
    password='',
    host='localhost',
    port='5432'
)
cursor = conn.cursor()

# Count tables in the 'bronze' schema
cursor.execute("""
    SELECT COUNT(*) 
    FROM information_schema.tables
    WHERE table_schema = 'Bronze'
""")
table_count = cursor.fetchone()[0]
print(f"✅ Total tables in Bronze schema: {table_count}")

cursor.close()
conn.close()


# Bulk Inserting into all Tables

In [ ]:


conn = engine.raw_connection()  # raw connection for COPY
cursor = conn.cursor()

print("✅ Connected successfully to PostgreSQL")

# =============================================
# 2️⃣ Logging setup
# =============================================
log_file = "PostgreSQL_Bronze_load_log_pg_resumable.txt"
success_count = 0
failure_count = 0
if os.path.exists(log_file):
    os.remove(log_file)

# =============================================
# 3️⃣ Get existing tables in Bronze schema
# =============================================
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema='Bronze'
""")
tables_loaded = [row[0] for row in cursor.fetchall()]
print(f"Tables already in Bronze schema: {len(tables_loaded)}")

# =============================================
# 4️⃣ Loop through CSVs and COPY into PostgreSQL
# =============================================
for i, file_path in enumerate(file_paths, 1):
    # PostgreSQL prefers lowercase table names, we convert CSV names
    table_name = os.path.basename(file_path).replace('.csv','').replace('.', '_').replace('-', '_').lower()
    
    # Full table reference with proper schema case
    full_table_name = f'"Bronze"."{table_name}"'

    # Check if table exists
    if table_name in tables_loaded:
        try:
            cursor.execute(f'SELECT COUNT(*) FROM {full_table_name}')
            rows_loaded = cursor.fetchone()[0]
        except Exception:
            rows_loaded = 0
    else:
        print(f"[{i}/{len(file_paths)}] ⚠️ Table {table_name} does not exist in schema Bronze, skipping")
        failure_count += 1
        with open(log_file, "a", encoding="utf-8") as f_log:
            f_log.write(f"❌ FAILED: {file_path} -> Table does not exist\n")
        continue

    # Count total rows in CSV (skip header)
    try:
        total_rows = sum(1 for _ in open(file_path, 'r', encoding='utf-8', errors='ignore')) - 1
    except Exception as e:
        failure_count += 1
        with open(log_file, "a", encoding="utf-8") as f_log:
            f_log.write(f"❌ FAILED: {file_path} -> {e}\n")
        print(f"❌ FAILED: {table_name} -> {e}")
        continue

    # Skip fully loaded tables
    if rows_loaded >= total_rows:
        print(f"[{i}/{len(file_paths)}] ⏭ Skipping {table_name} (already fully loaded)")
        continue
    elif rows_loaded > 0:
        print(f"[{i}/{len(file_paths)}] ⚡ Resuming {table_name} from row {rows_loaded + 2}...")

        # Create temp CSV with remaining rows
        temp_file = tempfile.NamedTemporaryFile(delete=False, mode='w', newline='', encoding='utf-8')
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            for j, line in enumerate(f):
                if j > rows_loaded:
                    temp_file.write(line)
        temp_file.close()
        bulk_file_path = temp_file.name
    else:
        print(f"[{i}/{len(file_paths)}] 🚀 Loading {table_name} ...")
        bulk_file_path = file_path

    # COPY into PostgreSQL
    try:
        with open(bulk_file_path, 'r', encoding='utf-8', errors='ignore') as f:
            cursor.copy_expert(f'COPY {full_table_name} FROM STDIN WITH CSV HEADER', f)
        conn.commit()

        success_count += 1
        with open(log_file, "a", encoding="utf-8") as f_log:
            f_log.write(f"✅ SUCCESS: {table_name}\n")

        print(f"✅ Loaded {table_name}")

        # Remove temporary file if used
        if rows_loaded > 0:
            os.remove(temp_file.name)

    except Exception as e:
        failure_count += 1
        conn.rollback()
        with open(log_file, "a", encoding="utf-8") as f_log:
            f_log.write(f"❌ FAILED: {file_path} -> {e}\n")
        print(f"❌ FAILED: {table_name} -> {e}")

# =============================================
# 5️⃣ Summary
# =============================================
print("\n🎉 Bronze Layer Ingestion Complete!")
print(f"✅ Successful files: {success_count}")
print(f"❌ Failed files: {failure_count}")
print(f"📄 Full log: {log_file}")

cursor.close()
conn.close()
